# Chapter 8: Stochastic Methods

In [2]:
from copy import copy
import jax
import numpy as np
from scipy import stats

## Algorithm 8.1

In [ ]:
class NoisyDescent:

    def __init__(self, submethod, sigma):
        self.submethod = submethod
        self.sigma = sigma
        self.k = 1
    
    def step(self, f, gradient, x):
        x = self.submethod.step(f, gradient, x)
        sigma = self.sigma[self.k]
        x += sigma*np.random.randn(x.shape[0])
        self.k += 1

        return x

### Example

In [19]:
class GradientDescent:
    def __init__(self, alpha):
        self.alpha = alpha # α

    def step(self, f, gradient, x):
        g = gradient(x)
        return x - self.alpha*g

def fun(x):
    return jax.numpy.sin(x[0]*x[1])+jax.numpy.exp(x[1]+x[2])-x[2]

x_0 = np.array([1.0, 2.0, 3.0])
gradient = jax.grad(fun)
submethod = GradientDescent(0.1)
sigma = [1.1/k for k in range(1,21)]
M = NoisyDescent(submethod, sigma)
x_1 = M.step(fun, gradient, x_0) 
print(x_1)

[  1.3609072 -12.642112  -12.549556 ]


## Algorithm 8.2

In [34]:
def rand_positive_spanning_set(alpha, n):
    delta = np.round(1/np.sqrt(alpha))
    L = np.diag(delta*np.random.choice([-1,1],size=n ))
    for i in range(0,n-1):
        for j in range(i+1,n):
            L[j,i] = np.random.uniform(-delta+1,delta-1)
    D = L[np.random.permutation(n),:]
    D = D[:, np.random.permutation(n)]
    D = np.concatenate([D, -np.sum(D,axis=1).reshape(-1,1)], axis=1)
    return [D[:,i] for i in range(0, n+1)]

## Algorithm 8.3

In [35]:
def mesh_adaptive_direct_search(f, x, epsilon):
    alpha, y, n = 1, f(x), x.shape[0]
    while alpha > epsilon:
        improved = False
        for i, d in enumerate(rand_positive_spanning_set(alpha, n)):
            x_prime = x + alpha*d
            y_prime = f(x_prime)
            if y_prime < y:
                x, y, improved = x_prime, y_prime, True
                x_prime = x + 3*alpha*d
                y_prime = f(x_prime)
                if y_prime < y:
                    x, y = x_prime, y_prime
        alpha = np.min([4*alpha,1]) if improved else alpha/4
    return x

### Example

In [38]:
def fun(x):
    return -np.exp(-(x[0]*x[1] - 1.5)**2 -(x[1]-1.5)**2)
x_0 = np.array([2.75, 2.25])
epsilon = 0.01
x_sol = mesh_adaptive_direct_search(fun, x_0, epsilon)
print(x_sol)

[0.98191814 1.61222439]


## Algorithm 8.4

In [2]:
def simulated_annealing(f, x, T, t, k_max):
    y = f(x)
    x_best, y_best = x, y
    for k in range(k_max):
        x_prime = x + T.rvs()
        y_prime = f(x_prime)
        delta_y = y_prime - y
        if delta_y <= 0 or np.random.random() < np.exp(-delta_y/t(k)):
            x, y = x_prime, y_prime
        
        if y_prime < y_best:
            x_best, y_best = x_prime, y_prime
        
    
    return x_best

### Example

In [4]:
def fun(x):
    return -np.exp(-(x[0]*x[1] - 1.5)**2 -(x[1]-1.5)**2)
x_0 = np.array([2.75, 2.25])
k_max = 2000
t = lambda k : 500.0/(k+1)
T = stats.norm()
x_sol = simulated_annealing(fun, x_0, T, t, k_max)
print(x_sol)

[1.56344716 1.06344716]


## Algorithm 8.5

In [35]:
def corona_update(v, a, c, ns):
    for i in range(v.shape[0]):
        ai, ci = a[i], c[i]
        if ai > 0.6*ns:
            v[i] *= (1 + ci*(ai/ns - 0.6)/0.4)
        elif ai < 0.4*ns:
            v[i] /= (1 + ci*(0.4-ai/ns)/0.4)

    return v

## Algorithm 8.6

In [53]:
def adaptive_simulated_annealing(f, x, v, t, epsilon, ns=20, nepsilon=4, gamma =0.85):
    nt = np.max([100, 5*x.shape[0]])
    c = np.zeros(x.shape[0])
    c.fill(2)
    y = f(x)
    x_best, y_best = x, y
    y_arr, n, U = [], x.shape[0], stats.uniform(-1,2)
    a, counts_cycles, counts_resets = np.zeros(n), 0, 0

    while True:
        for i in range(n):
            print(np.array(basis(i,n))*U.rvs()*v[i])
            x_prime = x + np.array(basis(i,n))*U.rvs()*v[i]
            y_prime = f(x_prime)
            delta_y = y_prime - y
            if delta_y < 0 or np.random.random() < np.exp(-delta_y/t):
                x, y = x_prime, y_prime
                a[i] += 1
                if y_prime < y_best:
                    x_best, y_best = x_prime, y_prime
            
        counts_cycles += 1
        if counts_cycles >= ns:
            counts_cycles = 0
            v = corona_update(v, a, c, ns)
            a.fill(0)
            counts_resets += 1

            if counts_resets >= nt:
                t *= gamma
                counts_resets = 0
                y_arr.insert(0,y)

                if not ( (len(y_arr) > nepsilon) and (y_arr[-1] - y_best <= epsilon) and np.all( [ np.abs(y_arr[-1]-y_arr[-1-u]) <= epsilon for u in range(nepsilon)]) ):
                    x, y = x_best, y_best
                else:
                    break
    return x_best

## Algorithm 8.7

In [13]:
def cross_entropy_method(f, P, k_max, m=100, m_elite=10):
    '''
    The Julia code prototype as is can not be implemented in Python
    '''
    for k in range(k_max):
        samples = P.rvs(m)
        order = np.argsort([f(samples[i]) for i in range(m)])
        P.fit(samples[order[:m_elite]]) #Currently scipy does not support the "fit" function for multivariate distributions
    return P

## Algorithm 8.8

In [ ]:
def natural_evolution_strategies(f, theta, k_max, m =100, alpha=0.01):
    '''
    The Julia code prototype as is can not be implemented in Python
    '''
    for k in range(k_max):
        samples = [theta.rvs() for i in range(m)]
        theta = update(theta, samples, m, alpha) #Needs to know the log likehood function of the distribution 'theta'
    return theta

## Algorithm 8.9

In [164]:
def covariance_matrix_adaptatiton(f, x, k_max, sigma=1.0, m=None, m_elite=None):
    '''
    Problems with computing Σ^-0.5
    '''
    if m == None:
        m = 4 + int(np.floor(3*np.log(x.shape[0])))
    
    if m_elite == None:
        m_elite = int(np.divide(m,2))

    mu, n = copy(x), x.shape[0]
    ws = np.repeat(np.log((m+1)/2),m) - np.log(np.arange(1,m+1,1) )
    ws[:m_elite] /= np.sum(ws[:m_elite])
    mu_eff = 1 / np.sum(ws[:m_elite]**2)
    csigma = (mu_eff + 2)/(n + mu_eff + 5)
    dsigma = 1 + 2*np.max([0, np.sqrt((mu_eff-1)/(n+1))-1]) + csigma
    cSigma = ( 4 + mu_eff/n ) / ( n + 4 + 2*mu_eff/n)
    c1 = 2 / ((n+1.3)*2 + mu_eff)
    cmu = np.min([1- c1, 2*(mu_eff-2+1/mu_eff)/((n+2)**2 + mu_eff) ])
    ws[m_elite:] *= - (1+c1/cmu)/np.sum(ws[m_elite:])
    E = n**0.5*(1-1/(4*n)+1/(21*n**2))
    psigma, pSigma, Sigma = np.zeros(n), np.zeros(n), np.identity(n)

    for k in range(k_max):

        P = stats.multivariate_normal(mean=mu, cov=sigma**2*Sigma)
        xs = [ P.rvs() for i in range(m) ]
        ys = [ f(x) for x in xs ]
        is_ = np.argsort(ys)

        deltas = [ (x-mu)/sigma for x in xs ]
        deltaw = np.sum( [ ws[i]*deltas[is_[i]] for i in range(m_elite) ]) 
        mu += sigma*deltaw
        C = Sigma**(-0.5)
        mask = np.ma.masked_equal(C, np.infty)
        C = mask.filled(fill_value=0)

        psigma = (1-csigma)*psigma + np.sqrt(csigma*(2-csigma)*mu_eff)*np.dot(C,np.repeat(deltaw,n,axis=0))
        sigma *= np.exp(csigma/dsigma * ( np.linalg.norm(psigma)/E-1) )
        hsigma = np.array([ int(np.linalg.norm(psigma[i])/np.sqrt(1-(1-csigma)**(2*k)) < (1.4+2/(n+1))*E) for i in range(n)])
        pSigma = (1 - cSigma)*pSigma + hsigma*np.sqrt(cSigma*(2-cSigma)*mu_eff)*deltaw

        w0 = [ ws[i] if ws[i] >= 0 else n*ws[i]/np.linalg.norm(C*deltas[is_[i]])**2 for i in range(m) ]
        Sigma = (1 - c1 - cmu)*Sigma + \
                c1*(np.dot(pSigma,pSigma) + (1-hsigma) * cSigma*(2-cSigma) * cSigma ) + \
                cmu*np.sum( [w0[i]*np.dot(deltas[is_[i]],deltas[is_[i]])  for i in range(m)] )
        
        Sigma = np.triu(Sigma) + np.triu(Sigma,1).T

    return mu